In [ ]:
# Fine-Tuning Data Preparation
## RAG vs Fine-Tuning: A Comparative Study for Legal QA

This notebook prepares the Indian Legal dataset for fine-tuning the Mistral model.

**Dataset**: [ninadn/indian-legal](https://huggingface.co/datasets/ninadn/indian-legal)  
**Model**: Mistral-7B-Instruct-v0.1  
**Task**: Legal Question Answering  
**Approach**: Instruction Tuning with QLoRA


In [ ]:
## 1. Setup and Imports


In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import json
import re
import os
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

# Create directories
os.makedirs('./processed_data', exist_ok=True)
os.makedirs('./models', exist_ok=True)

print("📦 Environment setup complete!")
print("🔍 Ready to process Indian Legal dataset for Mistral fine-tuning")


In [ ]:
## 2. Load Indian Legal Dataset from Hugging Face


In [ ]:
# Load the Indian Legal dataset from Hugging Face
print("🔄 Loading Indian Legal Dataset from Hugging Face...")
try:
    dataset = load_dataset("ninadn/indian-legal")
    print(f"✅ Dataset loaded successfully!")
    print(f"📊 Dataset structure: {dataset}")
    
    # Convert to pandas for analysis
    train_df = pd.DataFrame(dataset['train'])
    test_df = pd.DataFrame(dataset['test'])
    
    print(f"\n📈 Dataset Statistics:")
    print(f"  Training samples: {len(train_df):,}")
    print(f"  Test samples: {len(test_df):,}")
    print(f"  Total samples: {len(train_df) + len(test_df):,}")
    print(f"  Columns: {list(train_df.columns)}")
    
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    print("Please check your internet connection and Hugging Face access")
    raise


In [ ]:
## 3. Exploratory Data Analysis


In [ ]:
# Analyze the text data
print("📊 Analyzing Legal Documents...")

# Calculate text statistics
train_df['text_length'] = train_df['Text'].str.len()
test_df['text_length'] = test_df['Text'].str.len()

# Display basic statistics
print("\n📏 Text Length Statistics:")
print("=" * 50)
stats_data = {
    'Dataset': ['Training', 'Test'],
    'Count': [len(train_df), len(test_df)],
    'Mean Length': [train_df['text_length'].mean(), test_df['text_length'].mean()],
    'Median Length': [train_df['text_length'].median(), test_df['text_length'].median()],
    'Min Length': [train_df['text_length'].min(), test_df['text_length'].min()],
    'Max Length': [train_df['text_length'].max(), test_df['text_length'].max()],
    'Std Dev': [train_df['text_length'].std(), test_df['text_length'].std()]
}

stats_df = pd.DataFrame(stats_data)
print(stats_df.round(2).to_string(index=False))

# Sample documents
print(f"\n📝 Sample Legal Documents:")
print("=" * 80)
for i, (idx, row) in enumerate(train_df.head(2).iterrows()):
    print(f"\n🏛️ Document {i+1} ({len(row['Text'])} characters):")
    print("-" * 40)
    # Show first 600 characters
    preview = row['Text'][:600].replace('\n', ' ').strip()
    print(f"{preview}...")
    print("-" * 40)


In [ ]:
# Visualize text length distributions
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Indian Legal Dataset - Text Length Analysis', fontsize=16, fontweight='bold')

# Training set histogram
axes[0, 0].hist(train_df['text_length'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title(f'Training Set Distribution (n={len(train_df):,})')
axes[0, 0].set_xlabel('Text Length (characters)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(train_df['text_length'].mean(), color='red', linestyle='--', 
                   label=f'Mean: {train_df["text_length"].mean():.0f}')
axes[0, 0].legend()

# Test set histogram  
axes[0, 1].hist(test_df['text_length'], bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0, 1].set_title(f'Test Set Distribution (n={len(test_df):,})')
axes[0, 1].set_xlabel('Text Length (characters)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(test_df['text_length'].mean(), color='red', linestyle='--',
                   label=f'Mean: {test_df["text_length"].mean():.0f}')
axes[0, 1].legend()

# Box plot comparison
box_data = [train_df['text_length'], test_df['text_length']]
box_plot = axes[1, 0].boxplot(box_data, labels=['Training', 'Test'], patch_artist=True)
box_plot['boxes'][0].set_facecolor('skyblue')
box_plot['boxes'][1].set_facecolor('lightcoral')
axes[1, 0].set_title('Text Length Comparison')
axes[1, 0].set_ylabel('Text Length (characters)')

# Log scale for better visualization of distribution
axes[1, 1].hist(train_df['text_length'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 1].set_yscale('log')
axes[1, 1].set_title('Training Set - Log Scale')
axes[1, 1].set_xlabel('Text Length (characters)')
axes[1, 1].set_ylabel('Frequency (log scale)')

plt.tight_layout()
plt.show()

# Print percentiles for better understanding
print(f"\n📊 Training Set Text Length Percentiles:")
percentiles = [10, 25, 50, 75, 90, 95, 99]
for p in percentiles:
    val = np.percentile(train_df['text_length'], p)
    print(f"  {p}th percentile: {val:.0f} characters")


In [ ]:
## 4. Generate Question-Answer Pairs for Legal Documents


In [ ]:
# Fine-Tuning Data Preparation
## RAG vs Fine-Tuning: A Comparative Study

This notebook prepares the Indian Legal dataset for fine-tuning the Mistral model.

**Dataset**: ninadn/indian-legal
**Model**: Mistral-7B
**Task**: Legal Question Answering


In [ ]:
## 1. Setup and Imports


In [ ]:
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
import json
import re
import os
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create directories
os.makedirs('./processed_data', exist_ok=True)


In [ ]:
## 2. Load and Explore Dataset


In [ ]:
# Load the Indian Legal dataset
print("Loading Indian Legal Dataset...")
dataset = load_dataset("ninadn/indian-legal")
print(f"Dataset loaded successfully!")
print(f"Dataset structure: {dataset}")

# Convert to pandas for easier manipulation
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

print(f"\nTrain set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")
print(f"\nColumns: {train_df.columns.tolist()}")


In [ ]:
# Display sample data
print("Sample data from training set:")
print("=" * 50)
for i in range(2):
    print(f"\n--- Sample {i+1} ---")
    print(f"Text length: {len(train_df.iloc[i]['Text'])} characters")
    print(f"First 500 characters:")
    print(train_df.iloc[i]['Text'][:500] + "...")
    print("\n" + "="*50)


In [ ]:
## 3. Data Analysis and Statistics


In [ ]:
# Analyze text lengths
train_df['text_length'] = train_df['Text'].str.len()
test_df['text_length'] = test_df['Text'].str.len()

# Basic statistics
print("Text Length Statistics:")
print("=" * 30)
print(f"Train set:")
print(f"  Mean: {train_df['text_length'].mean():.2f} characters")
print(f"  Median: {train_df['text_length'].median():.2f} characters")
print(f"  Min: {train_df['text_length'].min()} characters")
print(f"  Max: {train_df['text_length'].max()} characters")
print(f"  Std: {train_df['text_length'].std():.2f} characters")

print(f"\nTest set:")
print(f"  Mean: {test_df['text_length'].mean():.2f} characters")
print(f"  Median: {test_df['text_length'].median():.2f} characters")
print(f"  Min: {test_df['text_length'].min()} characters")
print(f"  Max: {test_df['text_length'].max()} characters")
print(f"  Std: {test_df['text_length'].std():.2f} characters")


In [ ]:
# Visualize text length distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Text Length Analysis', fontsize=16, fontweight='bold')

# Histogram for train set
axes[0, 0].hist(train_df['text_length'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Train Set - Text Length Distribution')
axes[0, 0].set_xlabel('Text Length (characters)')
axes[0, 0].set_ylabel('Frequency')

# Histogram for test set
axes[0, 1].hist(test_df['text_length'], bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('Test Set - Text Length Distribution')
axes[0, 1].set_xlabel('Text Length (characters)')
axes[0, 1].set_ylabel('Frequency')

# Box plot comparison
box_data = [train_df['text_length'], test_df['text_length']]
axes[1, 0].boxplot(box_data, labels=['Train', 'Test'])
axes[1, 0].set_title('Text Length Comparison')
axes[1, 0].set_ylabel('Text Length (characters)')

# Log scale histogram for better visualization
axes[1, 1].hist(train_df['text_length'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 1].set_yscale('log')
axes[1, 1].set_title('Train Set - Text Length (Log Scale)')
axes[1, 1].set_xlabel('Text Length (characters)')
axes[1, 1].set_ylabel('Frequency (log scale)')

plt.tight_layout()
plt.show()


In [ ]:
## 4. Text Processing and QA Pair Generation


In [ ]:
def clean_legal_text(text):
    """
    Clean and preprocess legal text for fine-tuning
    """
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove special characters but keep legal punctuation
    text = re.sub(r'[^\w\s.,;:()\[\]"\'"-]', '', text)
    
    # Normalize quotes
    text = re.sub(r'[""'']', '"', text)
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text

def create_qa_pairs(text, max_length=1500):
    """
    Create question-answer pairs from legal text
    """
    qa_pairs = []
    
    # Question templates for legal documents
    question_templates = [
        "What is the main legal issue discussed in this case?",
        "What are the key provisions and clauses mentioned?",
        "What is the court's decision or ruling?",
        "What are the relevant legal sections cited?",
        "What are the rights and obligations of the parties?",
        "What legal principles or precedents are discussed?",
        "What are the terms and conditions mentioned?",
        "What penalties or consequences are discussed?"
    ]
    
    # Clean the text
    cleaned_text = clean_legal_text(text)
    
    # If text is too long, create multiple chunks
    if len(cleaned_text) > max_length:
        sentences = cleaned_text.split('. ')
        current_chunk = ""
        
        for sentence in sentences:
            if len(current_chunk + sentence) <= max_length:
                current_chunk += sentence + ". "
            else:
                if current_chunk:
                    # Create QA pairs for this chunk
                    for i, question in enumerate(question_templates[:3]):  # Limit to 3 per chunk
                        qa_pairs.append({
                            'question': question,
                            'context': current_chunk.strip(),
                            'answer': current_chunk.strip()
                        })
                current_chunk = sentence + ". "
        
        # Handle remaining chunk
        if current_chunk:
            for i, question in enumerate(question_templates[:3]):
                qa_pairs.append({
                    'question': question,
                    'context': current_chunk.strip(),
                    'answer': current_chunk.strip()
                })
    else:
        # Create QA pairs for the entire text
        for i, question in enumerate(question_templates[:4]):  # More questions for shorter texts
            qa_pairs.append({
                'question': question,
                'context': cleaned_text,
                'answer': cleaned_text
            })
    
    return qa_pairs

# Apply cleaning and generate QA pairs
print("Processing legal documents and generating QA pairs...")
all_qa_pairs = []

# Process a subset for initial development
sample_size = min(100, len(train_df))  # Start with 100 documents
for idx, row in tqdm(train_df.head(sample_size).iterrows(), total=sample_size, desc="Processing documents"):
    qa_pairs = create_qa_pairs(row['Text'])
    all_qa_pairs.extend(qa_pairs)

print(f"Generated {len(all_qa_pairs)} question-answer pairs from {sample_size} documents")


In [ ]:
## 5. Format Data for Instruction Tuning


In [ ]:
# Load tokenizer to check token lengths
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def format_for_instruction_tuning(qa_pair):
    """
    Format QA pair for instruction tuning with Mistral
    """
    # Truncate context and answer to reasonable lengths
    context = qa_pair['context'][:1000] if len(qa_pair['context']) > 1000 else qa_pair['context']
    answer = qa_pair['answer'][:800] if len(qa_pair['answer']) > 800 else qa_pair['answer']
    
    prompt = f"""<s>[INST] You are a legal assistant specializing in Indian law. Answer the following question based on the provided legal text.

Question: {qa_pair['question']}

Legal Text: {context} [/INST]

Based on the legal text provided, {answer}</s>"""
    
    return prompt

# Format all QA pairs
print("Formatting data for instruction tuning...")
formatted_examples = []

for qa_pair in tqdm(all_qa_pairs[:500], desc="Formatting examples"):  # Limit for initial testing
    formatted_text = format_for_instruction_tuning(qa_pair)
    
    # Check token length
    tokens = tokenizer.tokenize(formatted_text)
    if len(tokens) <= 2048:  # Limit to model's context length
        formatted_examples.append({
            'text': formatted_text,
            'token_count': len(tokens)
        })

print(f"Created {len(formatted_examples)} formatted examples for fine-tuning")

# Analyze token distributions
if formatted_examples:
    token_counts = [ex['token_count'] for ex in formatted_examples]
    print(f"\nToken count statistics:")
    print(f"  Mean: {np.mean(token_counts):.2f}")
    print(f"  Median: {np.median(token_counts):.2f}")
    print(f"  Max: {np.max(token_counts)}")
    print(f"  Min: {np.min(token_counts)}")
else:
    print("No examples created - check tokenizer setup")


In [ ]:
## 6. Split and Save Processed Data


In [ ]:
# Split formatted examples into train/validation
if formatted_examples:
    train_texts = [ex['text'] for ex in formatted_examples]
    train_split, val_split = train_test_split(train_texts, test_size=0.15, random_state=42)
    
    print(f"Training examples: {len(train_split)}")
    print(f"Validation examples: {len(val_split)}")
    
    # Create datasets
    train_dataset = Dataset.from_dict({'text': train_split})
    val_dataset = Dataset.from_dict({'text': val_split})
    
    # Save datasets
    train_dataset.save_to_disk('./processed_data/train')
    val_dataset.save_to_disk('./processed_data/val')
    
    print("\nDatasets saved successfully!")
    
    # Save some metadata
    metadata = {
        'total_examples': len(formatted_examples),
        'train_examples': len(train_split),
        'val_examples': len(val_split),
        'avg_token_length': np.mean(token_counts) if 'token_counts' in locals() else 0,
        'max_token_length': np.max(token_counts) if 'token_counts' in locals() else 0,
        'model_name': MODEL_NAME,
        'dataset_source': 'ninadn/indian-legal',
        'original_documents_processed': sample_size if 'sample_size' in locals() else 0
    }
    
    with open('./processed_data/metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print("Metadata saved!")
else:
    print("No formatted examples to save. Check data processing steps.")


# Display sample formatted examples
if formatted_examples:
    print("Sample Formatted Examples for Fine-tuning:")
    print("=" * 60)
    
    for i in range(min(2, len(formatted_examples))):
        print(f"\n--- Example {i+1} ---")
        print(f"Token count: {formatted_examples[i]['token_count']}")
        print("Content preview:")
        print(formatted_examples[i]['text'][:800] + "...")
        print("\n" + "=" * 60)


In [ ]:
## Summary

This notebook has successfully:

✅ **Loaded and analyzed** the Indian Legal dataset (ninadn/indian-legal)  
✅ **Generated question-answer pairs** suitable for legal QA  
✅ **Formatted data** for instruction tuning with Mistral  
✅ **Split data** into training and validation sets  
✅ **Saved processed datasets** for fine-tuning  

**Next Steps**: 
- Proceed to `2_fine_tuning.ipynb` to train the Mistral model
- The processed data is ready for efficient fine-tuning with LoRA/QLoRA

**Note**: This notebook processes a subset of the data for development. Increase `sample_size` for full dataset processing.
